In [22]:
import folium

In [5]:
import json
import pandas as pd

In [31]:
m = folium.Map([-38.043995, 145.264296], zoom_start=9)

df = pd.read_csv('../data/curated/property.csv')

for index, row in df.iterrows():
    lat = float(row['coordinates'][0])
    long = float(row['coordinates'][1])
    folium.Marker([lat, long]).add_to(m)

m

[-37.8860233, 145.0173065]


ValueError: could not convert string to float: '['